In [2]:
import statsapi
import pandas as pd
import numpy as np

<h3>Schedule API</h3>

In [94]:
sched = statsapi.schedule(start_date='01/01/2023',end_date='12/31/2024')
sched_df = pd.DataFrame(sched)
sched_df.to_csv('./test_output/schedule.csv', index=False)

<h3>Team API</h3>

In [16]:
def extract_field_from_json(json_obj, field_name):
    return None if field_name not in json_obj else json_obj[field_name]


def parse_json_field(json_obj, json_keys):
    # If no dictionary provide in row, return None values
    if pd.isna(json_obj):
        return pd.Series([None]*len(json_keys))
    
    value_list = []
    for json_key in json_keys:
        value_list.append(extract_field_from_json(json_obj, json_keys)) 

    return pd.Series(value_list)


# Retrieve team data from API
teams = statsapi.get('teams', params={'ver': 'v1'})['teams']
teams_df = pd.DataFrame(teams)

# Create list of fields that contain dictionaries as values in the form {'id', ... , 'name': ...}
# to extract data from
dict_fields = ['venue', 'league', 'division', 'sport', 'springLeague', 'springVenue']
for field in dict_fields:
    id_field_name = f'{field.lower()}_id'
    name_field_name = f'{field.lower()}_name'
    teams_df[[id_field_name, name_field_name]] = teams_df[field].apply(lambda x: parse_json_field(x, ['id', 'name']))

    teams_df.drop(columns=field, inplace=True)

# Only extract MLB teams
# Sport ID = 1 is the MLB
mlb_teams_df = teams_df[teams_df['sport_id'] == 1]

mlb_teams_df.to_csv('./test_output/teams.csv', index=False)

In [19]:
def parse_position_json(position_json):
    if pd.isna(position_json):
        return pd.Series([None, None, None, None])
    
    

team_roster_df = pd.DataFrame()

# for team_id in mlb_teams_df['id'].unique():

team_id = mlb_teams_df['id'].iloc[0]
team_roster = statsapi.get('team_roster', params={'ver': 'v1', 'teamId': team_id})['roster']
team_roster_df = pd.DataFrame(team_roster)
team_roster_df[['person_id', 'person_name']] = team_roster_df['person'].apply(lambda x: parse_id_name_json(x))



In [22]:
team_roster_df['position'].iloc[0]

{'code': '9', 'name': 'Outfielder', 'type': 'Outfielder', 'abbreviation': 'RF'}